In [1]:
import pandas as pd
import joblib
import json
from IPython.display import display, Markdown

def load_config(config_file="../config.json"):
    """Load configuration from a JSON file."""
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

In [2]:
test_df = pd.read_csv('../outputs/test_df.csv')

In [3]:
test_df

,projectid,total_price_excluding_optional_support,students_reached,fully_funded,date_posted,total_projects_in_city,total_projects_in_state,percentage_reached_month_1,percentage_reached_month_2,percentage_reached_month_3,...,month_posted_December,month_posted_February,month_posted_January,month_posted_July,month_posted_June,month_posted_March,month_posted_May,month_posted_November,month_posted_October,month_posted_September
0,a11d5c86692ef20e9f702094504bb5db,387.13,34.0,1.0,2013-01-01,290,126242,0.000000,0.000000,0.000000,...,False,False,True,False,False,False,False,False,False,False
1,8bdc6b4cfc850f6f7cc3963c48ba454b,1045.00,120.0,0.0,2013-01-01,172,2127,8.133971,8.133971,8.133971,...,False,False,True,False,False,False,False,False,False,False
2,88f263f1cd657bccce254ffc0b79df4d,868.02,28.0,1.0,2013-01-01,102,126242,100.000000,100.000000,100.000000,...,False,False,True,False,False,False,False,False,False,False
3,a8a6e08b6ae7ab1a88c16ba82ac8e9da,515.00,96.0,1.0,2013-01-01,206,14853,100.000000,100.000000,100.000000,...,False,False,True,False,False,False,False,False,False,False
4,be21151602e4fd47dd4012114fd519c6,238.56,25.0,1.0,2013-01-01,337,18615,17.815225,100.000000,100.000000,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114032,a0e839f24645e3d6dcbd327f8441b043,567.18,60.0,1.0,2013-12-31,20364,73182,53.289961,100.000000,100.000000,...,True,False,False,False,False,False,False,False,False,False
114033,f820ef3537f4445b0716244fae36f763,277.37,30.0,0.0,2013-12-31,55,2030,0.000000,0.000000,0.000000,...,True,False,False,False,False,False,False,False,False,False
114034,95ee208a51831edffa7cc2e0aa3e83cd,202.34,98.0,1.0,2013-12-31,2945,9837,100.000000,100.000000,100.000000,...,True,False,False,False,False,False,False,False,False,False
114035,1aa5e1d739a40c2763da1a0bb0f0f335,177.18,23.0,1.0,2013-12-31,855,17299,100.000000,100.000000,100.000000,...,True,False,False,False,False,False,False,False,False,False


In [4]:
test_df.columns

Index(['projectid', 'total_price_excluding_optional_support',
       'students_reached', 'fully_funded', 'date_posted',
       'total_projects_in_city', 'total_projects_in_state',
       'percentage_reached_month_1', 'percentage_reached_month_2',
       'percentage_reached_month_3', 'poverty_level_high poverty',
       'poverty_level_highest poverty', 'poverty_level_low poverty',
       'poverty_level_moderate poverty', 'school_metro_rural',
       'school_metro_suburban', 'school_metro_urban', 'grade_level_Grades 3-5',
       'grade_level_Grades 6-8', 'grade_level_Grades 9-12',
       'grade_level_Grades PreK-2', 'primary_focus_subject_Applied Sciences',
       'primary_focus_subject_Character Education',
       'primary_focus_subject_Civics & Government',
       'primary_focus_subject_College & Career Prep',
       'primary_focus_subject_Community Service', 'primary_focus_subject_ESL',
       'primary_focus_subject_Early Development',
       'primary_focus_subject_Economics',
       

In [5]:
test_df['date_posted'] = pd.to_datetime(test_df['date_posted'])

In [6]:
# Set max_date to filter projects posted for at least 3 months
fixed_max_date = pd.to_datetime('2013-12-31')
test_df['months_since_posted'] = ((fixed_max_date - test_df['date_posted']) / pd.Timedelta(days=30)).astype(int)
eligible_projects = test_df[test_df['months_since_posted'] >= 3].copy()

In [7]:
test_df.columns

Index(['projectid', 'total_price_excluding_optional_support',
       'students_reached', 'fully_funded', 'date_posted',
       'total_projects_in_city', 'total_projects_in_state',
       'percentage_reached_month_1', 'percentage_reached_month_2',
       'percentage_reached_month_3', 'poverty_level_high poverty',
       'poverty_level_highest poverty', 'poverty_level_low poverty',
       'poverty_level_moderate poverty', 'school_metro_rural',
       'school_metro_suburban', 'school_metro_urban', 'grade_level_Grades 3-5',
       'grade_level_Grades 6-8', 'grade_level_Grades 9-12',
       'grade_level_Grades PreK-2', 'primary_focus_subject_Applied Sciences',
       'primary_focus_subject_Character Education',
       'primary_focus_subject_Civics & Government',
       'primary_focus_subject_College & Career Prep',
       'primary_focus_subject_Community Service', 'primary_focus_subject_ESL',
       'primary_focus_subject_Early Development',
       'primary_focus_subject_Economics',
       

In [8]:
config = load_config()
poverty_levels = ["low", "moderate", "high", "highest"]
models = ["random_forest", "logistic_regression"]
recommendations = {}

In [9]:
recommendations = {}

poverty_levels = ["low", "moderate", "high", "highest"]
models = ["random_forest", "logistic_regression"]

for model_type in models:
    for pov_level in poverty_levels:
        pov_column = f"poverty_level_{pov_level} poverty"
        pov_projects = eligible_projects[eligible_projects[pov_column] == 1].copy()
        
        # Load the classifier
        classifier = joblib.load(f"../outputs/{model_type}_{pov_level}_poverty.pkl")
        
        X_test_filtered = pov_projects.drop(columns=['fully_funded', 'date_posted', 'months_since_posted', 'projectid'], errors='ignore')
        X_test_filtered = X_test_filtered.reindex(columns=classifier.feature_names_in_, fill_value=0)
        
        pov_projects[f'probability_fully_funded_{model_type}'] = classifier.predict_proba(X_test_filtered)[:, 1]
        
        pov_projects_sorted = pov_projects.sort_values(by=f'probability_fully_funded_{model_type}', ascending=False)

        if pov_level in ["high", "highest"]:
            top_recommendations = pov_projects_sorted.head(10)
        elif pov_level in ["low", "moderate"]:
            top_recommendations = pov_projects_sorted.head(3)

        recommendations[(model_type, pov_level)] = top_recommendations[['projectid', 'date_posted', f'probability_fully_funded_{model_type}']]

In [10]:
list(recommendations.keys())

[('random_forest', 'low'),
 ('random_forest', 'moderate'),
 ('random_forest', 'high'),
 ('random_forest', 'highest'),
 ('logistic_regression', 'low'),
 ('logistic_regression', 'moderate'),
 ('logistic_regression', 'high'),
 ('logistic_regression', 'highest')]

In [11]:
for key, recs in recommendations.items():
    model_type, pov_level = key  # Unpack manually
    # print(f"\nTop recommendations for {model_type.capitalize()} model, poverty level '{pov_level}':")
    display(key, recs)

('random_forest', 'low')

,projectid,date_posted,probability_fully_funded_random_forest
76,e94a5f08bb09a5c745edc4860b0d85ac,2013-01-01,1.0
57172,d97caf8b7243ba9128cc0a7aacdf9780,2013-08-30,1.0
55117,efd84a1f24856df0431a469a3abf4664,2013-08-25,1.0


('random_forest', 'moderate')

,projectid,date_posted,probability_fully_funded_random_forest
76264,0c61d6de55e1ec14bcf76540c132fee7,2013-10-02,1.0
23694,b313d50d7e4448569f988061630cb539,2013-04-14,1.0
56227,b7bf7820c5110ee7db3949b51a5a9c7d,2013-08-29,1.0


('random_forest', 'high')

...

,projectid,date_posted,probability_fully_funded_logistic_regression
19898,a0e2c86d61c3154e8c9f1fde77479016,2013-03-26,0.999954
44613,4c54491526dd79c04e1f5657ac5eed10,2013-07-30,0.999877
12685,5108480f02d6ca4dd6208d05ac5a7c91,2013-02-18,0.999322


('logistic_regression', 'high')

,projectid,date_posted,probability_fully_funded_logistic_regression
38828,9069d0603e76839c684dc6a1c138d639,2013-07-06,0.999955
25188,a80f26f1e5916b8d9dcb6bf36c5fee79,2013-04-25,0.999938
4167,504e1ec5da6aae62327e91989f50aa3c,2013-01-20,0.999871
22716,cf5c360adec5160846578246f7e65e5a,2013-04-12,0.999861
57062,00229f2003fd383c6ced36bc47761845,2013-08-30,0.999756
47462,10b65bc84faeab717d892ea2e0f7a9bd,2013-08-09,0.999698
62193,23093b3e4e0749f365c5637fd19258fa,2013-09-07,0.999587
23265,466c5a055c749e521c39c93a2f3d109d,2013-04-13,0.999578
40098,c22cf7e67f9ed001dcf1ccc687938aee,2013-07-13,0.999496
41722,10d40cb99f042abce156be2b62ea3395,2013-07-20,0.999413


('logistic_regression', 'highest')

,projectid,date_posted,probability_fully_funded_logistic_regression
13059,2be31e8ed070f2c71021cb7cd28f669b,2013-02-20,0.999996
59401,e891154767af72f5889a4ee9c269ccfc,2013-09-03,0.999884
11679,80de6d8194fedd2f27778aa58bb6d724,2013-02-15,0.999807
55716,11de69d170d074e3e5ec043a5ce73fe3,2013-08-27,0.999667
41104,815395153f4552eefefeecb873b06398,2013-07-16,0.999502
45722,81657e229e1dfa772abeee77b7693dc6,2013-08-04,0.999144
33104,6ecdac4a3ab7aa0f0f1641001f5d6401,2013-06-08,0.999135
22096,c9012494f0871fbe304cd4928189e2ea,2013-04-08,0.999123
44436,82af25a04bad1eed9a07f01c42881cc9,2013-07-29,0.999108
57521,a35d96eb60c1cb6eabecdd91709f4fb1,2013-08-31,0.999108


In [12]:
for key, recs in recommendations.items():
    model_type, pov_level = key  # Unpack model type and poverty level
    
    # Sort projects by probability to create a ranking
    recs_sorted = recs.sort_values(f'probability_fully_funded_{model_type}', ascending=False).reset_index(drop=True)
    recs_sorted['rank'] = recs_sorted.index + 1  # Add rank column
    
    # Add model type and poverty level columns
    recs_sorted['model_type'] = model_type.capitalize()
    recs_sorted['poverty_level'] = pov_level.capitalize()
    
    # Select only relevant columns and rename for consistency
    table = recs_sorted[['rank', 'projectid', 'model_type', 'poverty_level', f'probability_fully_funded_{model_type}']]
    table.rename(columns={f'probability_fully_funded_{model_type}': 'probability_fully_funded'}, inplace=True)
    
    # Display table with a header for context
    display(Markdown(f"### {model_type.capitalize()} Model - {pov_level.capitalize()} Poverty Level"))
    display(table)

/var/folders/5m/4tj5fx915fv1z073c22c2q7m0000gn/T/ipykernel_80504/692742374.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.rename(columns={f'probability_fully_funded_{model_type}': 'probability_fully_funded'}, inplace=True)


### Random_forest Model - Low Poverty Level

,rank,projectid,model_type,poverty_level,probability_fully_funded
0,1,e94a5f08bb09a5c745edc4860b0d85ac,Random_forest,Low,1.0
1,2,d97caf8b7243ba9128cc0a7aacdf9780,Random_forest,Low,1.0
2,3,efd84a1f24856df0431a469a3abf4664,Random_forest,Low,1.0


/var/folders/5m/4tj5fx915fv1z073c22c2q7m0000gn/T/ipykernel_80504/692742374.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.rename(columns={f'probability_fully_funded_{model_type}': 'probability_fully_funded'}, inplace=True)


### Random_forest Model - Moderate Poverty Level

...

### Logistic_regression Model - High Poverty Level

,rank,projectid,model_type,poverty_level,probability_fully_funded
0,1,9069d0603e76839c684dc6a1c138d639,Logistic_regression,High,0.999955
1,2,a80f26f1e5916b8d9dcb6bf36c5fee79,Logistic_regression,High,0.999938
2,3,504e1ec5da6aae62327e91989f50aa3c,Logistic_regression,High,0.999871
3,4,cf5c360adec5160846578246f7e65e5a,Logistic_regression,High,0.999861
4,5,00229f2003fd383c6ced36bc47761845,Logistic_regression,High,0.999756
5,6,10b65bc84faeab717d892ea2e0f7a9bd,Logistic_regression,High,0.999698
6,7,23093b3e4e0749f365c5637fd19258fa,Logistic_regression,High,0.999587
7,8,466c5a055c749e521c39c93a2f3d109d,Logistic_regression,High,0.999578
8,9,c22cf7e67f9ed001dcf1ccc687938aee,Logistic_regression,High,0.999496
9,10,10d40cb99f042abce156be2b62ea3395,Logistic_regression,High,0.999413


/var/folders/5m/4tj5fx915fv1z073c22c2q7m0000gn/T/ipykernel_80504/692742374.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.rename(columns={f'probability_fully_funded_{model_type}': 'probability_fully_funded'}, inplace=True)


### Logistic_regression Model - Highest Poverty Level

,rank,projectid,model_type,poverty_level,probability_fully_funded
0,1,2be31e8ed070f2c71021cb7cd28f669b,Logistic_regression,Highest,0.999996
1,2,e891154767af72f5889a4ee9c269ccfc,Logistic_regression,Highest,0.999884
2,3,80de6d8194fedd2f27778aa58bb6d724,Logistic_regression,Highest,0.999807
3,4,11de69d170d074e3e5ec043a5ce73fe3,Logistic_regression,Highest,0.999667
4,5,815395153f4552eefefeecb873b06398,Logistic_regression,Highest,0.999502
5,6,81657e229e1dfa772abeee77b7693dc6,Logistic_regression,Highest,0.999144
6,7,6ecdac4a3ab7aa0f0f1641001f5d6401,Logistic_regression,Highest,0.999135
7,8,c9012494f0871fbe304cd4928189e2ea,Logistic_regression,Highest,0.999123
8,9,82af25a04bad1eed9a07f01c42881cc9,Logistic_regression,Highest,0.999108
9,10,a35d96eb60c1cb6eabecdd91709f4fb1,Logistic_regression,Highest,0.999108


In [13]:
for (model_type, pov_level), top_recs in recommendations.items():
    top_recs = top_recs.merge(test_df, on='projectid', how='left', suffixes=('', '_test'))
    recommendations[(model_type, pov_level)] = top_recs
    display(recommendations)

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e94a5f08bb09a5c745edc4860b0d85ac  2013-01-01   
 1  d97caf8b7243ba9128cc0a7aacdf9780  2013-08-30   
 2  efd84a1f24856df0431a469a3abf4664  2013-08-25   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  fully_funded  \
 0                                  752.10              30.0           1.0   
 1                                  255.22             160.0           1.0   
 2                                  450.50              23.0           1.0   
 
   date_posted_test  total_projects_in_city  total_projects_in_state  \
 0       2013-01-01                     392                   126242   
 1       2013-08-30                     131                   126242   
 2       2013-08-25                     114   

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e94a5f08bb09a5c745edc4860b0d85ac  2013-01-01   
 1  d97caf8b7243ba9128cc0a7aacdf9780  2013-08-30   
 2  efd84a1f24856df0431a469a3abf4664  2013-08-25   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  fully_funded  \
 0                                  752.10              30.0           1.0   
 1                                  255.22             160.0           1.0   
 2                                  450.50              23.0           1.0   
 
   date_posted_test  total_projects_in_city  total_projects_in_state  \
 0       2013-01-01                     392                   126242   
 1       2013-08-30                     131                   126242   
 2       2013-08-25                     114   

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e94a5f08bb09a5c745edc4860b0d85ac  2013-01-01   
 1  d97caf8b7243ba9128cc0a7aacdf9780  2013-08-30   
 2  efd84a1f24856df0431a469a3abf4664  2013-08-25   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  fully_funded  \
 0                                  752.10              30.0           1.0   
 1                                  255.22             160.0           1.0   
 2                                  450.50              23.0           1.0   
 
   date_posted_test  total_projects_in_city  total_projects_in_state  \
 0       2013-01-01                     392                   126242   
 1       2013-08-30                     131                   126242   
 2       2013-08-25                     114   

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e94a5f08bb09a5c745edc4860b0d85ac  2013-01-01   
 1  d97caf8b7243ba9128cc0a7aacdf9780  2013-08-30   
 2  efd84a1f24856df0431a469a3abf4664  2013-08-25   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  fully_funded  \
 0                                  752.10              30.0           1.0   
 1                                  255.22             160.0           1.0   
 2                                  450.50              23.0           1.0   
 
   date_posted_test  total_projects_in_city  total_projects_in_state  \
 0       2013-01-01                     392                   126242   
 1       2013-08-30                     131                   126242   
 2       2013-08-25                     114   

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e94a5f08bb09a5c745edc4860b0d85ac  2013-01-01   
 1  d97caf8b7243ba9128cc0a7aacdf9780  2013-08-30   
 2  efd84a1f24856df0431a469a3abf4664  2013-08-25   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  fully_funded  \
 0                                  752.10              30.0           1.0   
 1                                  255.22             160.0           1.0   
 2                                  450.50              23.0           1.0   
 
   date_posted_test  total_projects_in_city  total_projects_in_state  \
 0       2013-01-01                     392                   126242   
 1       2013-08-30                     131                   126242   
 2       2013-08-25                     114   

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e94a5f08bb09a5c745edc4860b0d85ac  2013-01-01   
 1  d97caf8b7243ba9128cc0a7aacdf9780  2013-08-30   
 2  efd84a1f24856df0431a469a3abf4664  2013-08-25   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  fully_funded  \
 0                                  752.10              30.0           1.0   
 1                                  255.22             160.0           1.0   
 2                                  450.50              23.0           1.0   
 
   date_posted_test  total_projects_in_city  total_projects_in_state  \
 0       2013-01-01                     392                   126242   
 1       2013-08-30                     131                   126242   
 2       2013-08-25                     114   

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e94a5f08bb09a5c745edc4860b0d85ac  2013-01-01   
 1  d97caf8b7243ba9128cc0a7aacdf9780  2013-08-30   
 2  efd84a1f24856df0431a469a3abf4664  2013-08-25   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  fully_funded  \
 0                                  752.10              30.0           1.0   
 1                                  255.22             160.0           1.0   
 2                                  450.50              23.0           1.0   
 
   date_posted_test  total_projects_in_city  total_projects_in_state  \
 0       2013-01-01                     392                   126242   
 1       2013-08-30                     131                   126242   
 2       2013-08-25                     114   

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e94a5f08bb09a5c745edc4860b0d85ac  2013-01-01   
 1  d97caf8b7243ba9128cc0a7aacdf9780  2013-08-30   
 2  efd84a1f24856df0431a469a3abf4664  2013-08-25   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  fully_funded  \
 0                                  752.10              30.0           1.0   
 1                                  255.22             160.0           1.0   
 2                                  450.50              23.0           1.0   
 
   date_posted_test  total_projects_in_city  total_projects_in_state  \
 0       2013-01-01                     392                   126242   
 1       2013-08-30                     131                   126242   
 2       2013-08-25                     114   

In [14]:
stem_subjects = ['primary_focus_subject_Applied Sciences',
                 'primary_focus_subject_Health & Life Science',
                 'primary_focus_subject_Environmental Science',
                 'primary_focus_subject_Mathematics']

def classify_stem(row):
    # Check if any STEM subject column is 1
    return 'STEM' if row[stem_subjects].sum() > 0 else 'Non-STEM'

test_df['STEM_category'] = test_df.apply(classify_stem, axis=1)

In [15]:
for (model_type, pov_level), df in recommendations.items():
    df = df.drop_duplicates().reset_index(drop=True)  # Drop any existing duplicates
    df = df.reset_index(drop=True)
    recommendations[(model_type, pov_level)] = df.assign(model_type=model_type, pov_level=pov_level)

# Concatenate all DataFrames from recommendations
all_recs = pd.concat(recommendations.values(), ignore_index=True)

In [17]:
all_recs.columns

Index(['projectid', 'date_posted', 'probability_fully_funded_random_forest',
       'total_price_excluding_optional_support', 'students_reached',
       'fully_funded', 'date_posted_test', 'total_projects_in_city',
       'total_projects_in_state', 'percentage_reached_month_1',
       'percentage_reached_month_2', 'percentage_reached_month_3',
       'poverty_level_high poverty', 'poverty_level_highest poverty',
       'poverty_level_low poverty', 'poverty_level_moderate poverty',
       'school_metro_rural', 'school_metro_suburban', 'school_metro_urban',
       'grade_level_Grades 3-5', 'grade_level_Grades 6-8',
       'grade_level_Grades 9-12', 'grade_level_Grades PreK-2',
       'primary_focus_subject_Applied Sciences',
       'primary_focus_subject_Character Education',
       'primary_focus_subject_Civics & Government',
       'primary_focus_subject_College & Career Prep',
       'primary_focus_subject_Community Service', 'primary_focus_subject_ESL',
       'primary_focus_subject_

In [18]:
all_recs

,projectid,date_posted,probability_fully_funded_random_forest,total_price_excluding_optional_support,students_reached,fully_funded,date_posted_test,total_projects_in_city,total_projects_in_state,percentage_reached_month_1,...,month_posted_June,month_posted_March,month_posted_May,month_posted_November,month_posted_October,month_posted_September,months_since_posted,model_type,pov_level,probability_fully_funded_logistic_regression
0,e94a5f08bb09a5c745edc4860b0d85ac,2013-01-01,1.0,752.10,30.0,1.0,2013-01-01,392,126242,100.000000,...,False,False,False,False,False,False,12,random_forest,low,NaN
1,d97caf8b7243ba9128cc0a7aacdf9780,2013-08-30,1.0,255.22,160.0,1.0,2013-08-30,131,126242,100.000000,...,False,False,False,False,False,False,4,random_forest,low,NaN
2,efd84a1f24856df0431a469a3abf4664,2013-08-25,1.0,450.50,23.0,1.0,2013-08-25,114,7021,100.000000,...,False,False,False,False,False,False,4,random_forest,low,NaN
3,0c61d6de55e1ec14bcf76540c132fee7,2013-10-02,1.0,431.14,23.0,1.0,2013-10-02,1965,10716,0.000000,...,False,False,False,False,True,False,3,random_forest,moderate,NaN
4,b313d50d7e4448569f988061630cb539,2013-04-14,1.0,378.53,35.0,1.0,2013-04-14,357,126242,100.000000,...,False,False,False,False,False,False,8,random_forest,moderate,NaN
5,b7bf7820c5110ee7db3949b51a5a9c7d,2013-08-29,1.0,426.35,160.0,1.0,2013-08-29,11,126242,100.000000,...,False,False,False,False,False,False,4,random_forest,moderate,NaN
6,88f263f1cd657bccce254ffc0b79df4d,2013-01-01,1.0,868.02,28.0,1.0,2013-01-01,102,126242,100.000000,...,False,False,False,False,False,False,12,random_forest,high,NaN
7,4ebd0837defce812ff9d312c3f78db33,2013-08-04,1.0,542.62,25.0,1.0,2013-08-04,1428,43478,100.000000,...,False,False,False,False,False,False,4,random_forest,high,NaN
8,3fe2b20e3ab2c4f00d20a217329f2dc3,2013-08-04,1.0,156.05,25.0,1.0,2013-08-04,943,10469,100.000000,...,False,False,False,False,False,False,4,random_forest,high,NaN
9,cc861ce787eb2377f5525755c8d2f159,2013-08-04,1.0,224.29,23.0,1.0,2013-08-04,98,18615,0.000000,...,False,False,False,False,False,False,4,random_forest,high,NaN
